In [ ]:
import numpy as np

np.random.seed(2016)  # for reproducibility
np.set_printoptions(precision = 3)  # to print less digits for floats

## Εισαγωγή

**Καλώς ήρθατε στο εικονικό εργαστήριο!**

Διαβάστε τις οδηγίες στα κελιά κειμένου και εκτελέστε τα κελιά κώδικα για να δείτε τα αποτελέσματα.

Η αρίθμηση των ασκήσεων είναι η ίδια με αυτή του [βιβλίου](https://repository.kallipos.gr/handle/11419/6003)

*Μην ξεχάσετε να εκτελέσετε και το 1ο κελί πριν την Εισαγωγή!*

## Άσκηση 16

Σκοπός αυτής της άσκησης είναι να εξοικιωθείτε με την βιβλιοθήκη `simple_markov_chain_lib.py`.
Το αρχείο θα πρέπει να βρίσκεται στον ίδιο φάκελο με αυτό το notebook. Μπορείτε να το ανοίξετε και να το επεξεργαστείτε μέσω του browser. Δεν χρειάζεται να κατανοήσετε κάθε λεπτομέριες καθώς αυτό απαιτεί αρκετή εμπειρία με Python (και αντικειμενοστραφή προγραμματισμό). Θα το χρησιμοποιήσουμε απλά σαν βιβλιοθήκη που προσωμοιώνει μαρκοβιανές αλυσίδες.

Παρόλα αυτά, αν έχετε κάποια οικειότητα με την Python αξίζει να ρίξετε μια ματία και ειδικά στις "εσωτερικές μεθόδους" `_partial_sums` και `_next_state` (η κάτω παύλα `_` στην αρχή δηλώνει ότι είναι εσωτερικές μέθοδοι και δεν πρέπει να καλούνται από το χρήστη). Οι 2 αυτές μέθοδοι υλοποιούν την γεννήτρια τυχαίων αριθμών που περιγράφεται στην **άσκηση 10**.

### simple_markov_chain_lib

Το `simple_markov_chain_lib.py` περιέχει μόνο ένα "αντικείμενο" την κλάση `markov_chain`. 
Η επόμενη γραμμή το εισάγει στο περιβάλλον της Python


In [ ]:
from simple_markov_chain_lib import markov_chain  # import markov chain simulator

Σε αυτό το εργαστήριο θα δείτε πως μπορείτε να χρησιμοποιήσετε αυτή τη κλάση. Συγκεκριμένα θα εστιάσουμε σε 3 πράγματα:

1. Πως να δημιουργήσετε μία καινούργια αλυσίδα
2. Πώς να προσομοιώσετε έναν τυχαίο περίπατο στο χώρο καταστάσεων
3. Πώς να ελέγξετε τις ιδιότητες αυτής της αλυσίδας

#### Δημιουργία καινούργιας μακροβιανής αλυσίδας

Για να φτιάξετε μια καινούργια αλυσίδα χρειάζονται 2 ορίσματα:

1. `markov_table`: Ο πίνακας πιθανοτήτων μετάβασεις και
2. `init_dist` (προαιρετικά): η αρχική κατανομή

Επιδή οι πίνακες μετάβασεις είναι "αραιοί" τις περισσότερες φορές τα ορίσματα 
`markov_table` και `init_dist` δεν είναι πραγματικοί πίνακες αλλά "λεξικά" (`dict`). 
Τα λεξικά αυτά περιγράφουν μια συνάρτηση μάζας πιθανότητας (*σ.μ.π*) ως εξής:

* `key` = κατάσταση (δεν χρειάζεται να είναι αριθμός)
* `value` = πιθανότητα (πρέπει να είναι αριθμός στο διάστημα $[0, 1]$)

όσες καταστάσεις **δεν** περιέχονται στο λεξικό θεωρείται όχι έχουν μηδενική πιθανότητα.
Το άθροισμα όλων των πιθανοτήτων που περιέχονται στο λεξικό
πρέπει να είναι ίσο με 1 αλλιώς ο κώδικας θα εγείρει σφάλμα.

Για την **αρχική κατανομή** (`init_dist`),  καθε κλειδί (`key`) αντιστοιχεί σε μία κατάσταση και 
η τιμή (`value`) που έχει αυτό το κλειδί είναι η πιθανότητα η αλυσίδα να εκκινήσει από αυτή τη κατάσταση.

Ο **πίνακας πιθανοτήτων μετάβασης** είναι ένα σύνολο από *σ.μ.π*. 
Επομένως, το όρισμα `markov_table` ορίζεται σαν ένα *λεξικό από λεξικά*. 
Κάθε κλειδί (`key`) αντιστοιχεί σε μία γραμμή του πίνακα και 
η τιμή (`value`) που αντιστοιχεί σε αυτό το κλειδί είναι ένα λεξικό (*σ.μ.π*) 
όπου τα κλειδιά είναι οι πιθανές καταστάσεις μετάβασης 
και οι τιμές οι πιθανότητες μετάβασης.


Ο κώδικας στο επόμενο κελί είναι ένα παράδειγμα που υλοποιεί την αλυσίδα με πίνακα πιθανοτήτων μετάβασης

$$
P = 
\begin{pmatrix}
0   & 1/2 & 1/2 &  0  & 0 \\
1/3 &  0  &  0  & 2/3 & 0 \\
0   &  0  &  1  &  0  & 0 \\
1/2 &  0  &  0  & 1/2 & 0 \\
0   &  0  &  0  &  0  & 1 
\end{pmatrix}
$$

και αρχική κατάσταση την γραμμή 0 (η αρίθμηση στην Python ξεκινάει από το 0).

In [ ]:
# Initial Distribution
p = {0: 1.}  # we a.s. start from state 0 with probability 1

# Transition Table
P = {
    0: {1: .5, 2: .5},  # from state 0 we move to state 1 with prob 0.5 and to state 2 with 0.5
    1: {0: 1/3, 3: 2/3},
    2: {2: 1.},
    3: {0: .5, 3: .5},
    4: {4: 1.}
}

ex17 = markov_chain(P, p)

#### Simulate random walk

Ένα αντικείμενο `markov_chain` χαρακτηρίζεται από 2 κύριες ιδιότητες 
(εκτός από το `markov_table` και την αρχική κατανομή):

1. **`step`**: το παρών βήμα της αλυσίδας (το $n$ στο $\{X_n\}_{n \in \mathbb{N}_0}$)
2. **`running_state`**: την παρούσα κατάσταση ($X_n$)

Για να προσομοιώσουμε την αλυσίδα πρέπει αρχικά να την *εκκινήσουμε* (`start`) 
και στη συνέχεια να την *προχωρήσουμε* (`move`) στο χώρο τον καταστάσεων.

Το `markov_chain` έχει 2 μεθόδους για αυτό το σκοπό:

* **`start`**: θέτει το βήμα στο  `0` και επιλέγει μια τυχαία κατάσταση εκκίνησης σύμφωνα με την αρχική κατανομή
* **`move`**: επιλέγει την επόμενη κατάσταση και επαυξάνει το χρόνο (`step`) κατά ένα.


In [ ]:
# First step
ex17.start()
n, Xn = ex17.steps, ex17.running_state  # n = steps, Xn = running state
print("At time {} the chain is at state {}".format(n, Xn))

ex17.start()
n, Xn = ex17.steps, ex17.running_state
print("At time {} the chain is at state {}".format(n, Xn))

# Subsequent Steps
ex17.move()
n, Xn = ex17.steps, ex17.running_state
print("At time {} the chain is at state {}".format(n, Xn))

ex17.move()
n, Xn = ex17.steps, ex17.running_state
print("At time {} the chain is at state {}".format(n, Xn))


παρατηρείστε ότι όταν κάνουμε `start` για δεύτερη φορά ο χρόνος μένει στο 0 (η κατάσταση μπορεί να αλλάξει) ενώ όταν κάνουμε `move` ο χρόνος αυξάνεται (η κατάσταση μπορεί να μείνει και ίδια).

Ο παρακάτω κώδικας προσομοιώνει τα 10 πρώτα βήματα της αλυσίδας:

In [ ]:
ex17.start()
print("At time {} the chain is in state {}".format(ex17.steps, ex17.running_state))
for i in range(10):
    ex17.move()
    print("At time {} the chain is in state {}".format(ex17.steps, ex17.running_state))


Ξανατρέχτε το προηγούμενο κελί μερικές φορές και δείτε πως τα αποτελέσματα αλλάζουν.

#### Άλλες ιδιότητες  του `markov_chain`

* `probs_matrix` ο πίνακας πιθανοτήτων μετάβασης (σε μορφή πίνακα)
* `eigenvalues` οι ιδιοτιμές του `probs_matrix`
* `probs_state` η συνάρτηση μάζας πιθανότητας της αλυσίδας στο παρών βήμα (υπολογισμένη αναλυτικά).
* `communication_classes` ένα λεξικό με τις *ανοιχτές* (`open`) και *κλειστές* (`closed`) κλάσεις επικοινωνίας της αλυσίδας
* `len(markov_chain)`: το πληθάριθμος του χώρου καταστάσεων
* `init_dist`: η αρχική κατανομή (μπορείτε να την αλλάξετε άμα θέλετε)

In [ ]:
print("ex17.probs_matrix:")
print(ex17.probs_matrix)

print("\nex17.eigenvalues:")
print(np.round(ex17.eigenvalues, 2))

print("\nNumber of states: {}".format(len(ex17)))

print("\nInitial Distriubiton: {}".format(str(ex17.init_dist)))

print("\nState Distribution after {} steps".format(ex17.steps))
print(ex17.probs_state)

## Έξάσκηση

(**Άσκηση 17 του βιβλίου. Το πρόβλημα περιγράφεται στην άσκηση 11**)


Σε ένα ράφι της βιβλιοθήκης υπάρχουν 3 βιβλία:

* **A**lgebra
* **B**asic Topology
* **C**alculus

που θα συμβολίζουμε με `A, B, C`.

Κάθε πρωί παίρνετε τυχαία ένα βιβλίο από τη θέση του
με πιθανότητες p, q, r, αντίστοιχα.  ́Οταν τελειώνετε το διάβασμά σας για την ημέρα, το ξαναβάζετε στο
ράφι στην αριστερότερη θέση. Η διάταξη των βιβλίων είναι μια μαρκοβιανή αλυσίδα στον χώρο $\mathbb{X}$ των
μεταθέσεων των συμβόλων $\{A, B, C\}$.

Προσομοιώστε τα 20 πρώτα βήματα της αλυσίδας.

In [ ]:
# Your code goes here!